In [ ]:
import pandas as pd  
df_json = pd.read_json('/home/soukaina/Documents/dataset/BeCode/kone_classification.json')
df_json.head()

In [ ]:
df_json.shape

In [ ]:
df_json.duplicated()

In [ ]:
df_json.duplicated().count()

In [ ]:
df_json.head(50)

In [ ]:
df_json['text'].duplicated()

In [ ]:
print(df_json['label'].count())

In [ ]:
print(df_json['label'].max)

In [ ]:
df_json[df_json.culture == 'nl-nl'].shape[0]


In [ ]:
df_json[df_json.culture == 'fr-fr'].shape[0]


In [ ]:
df_json[df_json.label == 'OTHER'].shape[0]


In [ ]:
df_json[df_json.source == 'WORKFLOW'].shape[0]


In [ ]:
df_json[df_json.source == 'TRAINING'].shape[0]


In [ ]:
# df_json.drop(df_json.loc[df_json['source']=='WORKFLOW'].index)
df_json.drop(df_json.index[df_json['source'] == 'WORKFLOW'],inplace=True)

In [ ]:
df_json

In [ ]:
df_json.drop(df_json.index[df_json['culture'] == 'nl-nl'],inplace=True)

In [ ]:
df_json

In [ ]:
# df_json[df_json.duplicated()]
df_json[df_json.duplicated(['label','text'])]


In [ ]:
df_json['text']=df_json['text'].str.lower()
df_json

In [ ]:
import matplotlib.pyplot as plt

# visualize the data, and see how many numbers of text are there per label

fig = plt.figure(figsize=(15,10))
colors = ['grey','grey','grey','grey','grey','grey','grey','grey','grey',
    'grey','darkblue','darkblue','darkblue']
df_json.groupby('label').text.count().sort_values().plot.barh(
    ylim=0, color=colors, title= 'NUMBER OF texts IN EACH label')
plt.xlabel('Number of ocurrences', fontsize = 10)

In [ ]:
df_json['label'].nunique()

In [ ]:
df_json['label']

In [ ]:
df_json['length'] = df_json['text'].apply(lambda x: len(x))
df_json.head()

In [ ]:
df_json.length.max()


In [ ]:
import pandas as pd
import json
import re


In [ ]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.tokenizer import Tokenizer

In [ ]:
# some text cleaning functions
def convert_to_lower(text):
    return text.lower()

def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc

def remove_special_char(text):
    special_char = r'[^\w\s]|.:,*"'
    remove_special_char = re.sub(pattern=special_char, repl=" ", string=text)
    return remove_special_char

In [ ]:
df_json['text'] = df_json['text'].apply(lambda x: convert_to_lower(x))
df_json['text'] = df_json['text'].apply(lambda x: remove_numbers(x))
df_json['text'] = df_json['text'].apply(lambda x: remove_extra_white_spaces(x))
df_json['text'] = df_json['text'].apply(lambda x: remove_special_char(x))

In [ ]:
df_json['text'].iloc[50]


In [ ]:
df_json['length_after_cleaning'] = df_json['text'].apply(lambda x: len(x))
df_json.head()

In [ ]:
print(len(df_json['label'].tolist()))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# creating initial dataframe

# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df_json['label_encode'] = labelencoder.fit_transform(df_json['label'])
df_json

In [ ]:
print(len(df_json['label_encode'].tolist()))

# Augmentation

In [ ]:
import nlpaug.augmenter.word.context_word_embs as aug


In [ ]:
sample_text = df_json['text'].iloc[100]


In [ ]:
sample_text


In [ ]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")


In [ ]:
augmented_sample_text = augmenter.augment(sample_text)
augmented_sample_text


In [ ]:
for i in range(5):
    print(augmenter.augment(sample_text))

In [ ]:
value_count=df_json['label'].value_counts()
value_count.sort_values

In [ ]:
df_json.sort_values(by='label', inplace=True)
df_json

In [ ]:

df_minority=df_json.loc[['MR08','MR16','SH11','SH16','DOC03','SH07','MOD','CAR03','CAR01','CAR08','CAR23','CAR09','CAR04','CAR18','MR05','DOC04','SH19','LD13','CAR26','MR04','LD05','SH23','CAR12','MR07','OOS','LD01','SH02','SH26','LD16','CAR25','CAR19','LD11','DOC10','MR15','CAR15','SH03','DOC08','CAR16','SH14','LD14','CD02','DOC09','SH27','CW01'],['text','culture','document_id','annotation_id','source','length','length_after_cleaning','label_encode']]

In [ ]:
df_minority

In [ ]:
df_minority.reset_index(inplace=True)

In [ ]:
df_minority

In [ ]:
value_count1=df_minority['label'].value_counts()
value_count1

In [ ]:
# frames = [df_majority, df_minority]  
# df_merged = pd.concat(frames)

In [ ]:
df_minority.columns
  
df_minority['label'].nunique()

In [ ]:
df_minority.reset_index(drop=True)
df_minority

In [ ]:
from tqdm.auto import tqdm
from sklearn.utils import shuffle


In [ ]:
def augmentMyData(df, augmenter, repetitions=2, samples=30):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df.reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['text'].iloc[i])
            augmented_texts.append(augmented_text)
            
        
    data = {
        'label': df['label'].tolist(),
        'text': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
new_df = df_minority.drop(labels=['length', 'length_after_cleaning'], axis=1)
new_df.head()

In [ ]:
aug_df = augmentMyData(df_minority, augmenter,samples=30)


In [ ]:
aug_df['label'].value_counts()


In [ ]:
print("Original: ", result.shape)
print("Augmented: ", aug_df.shape)

In [ ]:
aug_df